# Projeto de Insights

## 1. Objetivo

- Gerar insight através da análise e manipulação dos dados para auxiliar a tomada de decisão do time de negócio


## 2. Metodologia

### 2.1. Problema de negócio

- Quais são os imóveis que a House Rocket deveria comprar e por qual preço
- Uma vez que o imóvel foi comprado, qual o melhor momento para revendê-lo e por qual preço

### 2.2. Planejamento da solução

- **Saída:**
    - Reposta:
        - Filtrar os imóveis com preço de custo abaixo da média da região.
        - Calcular a margem para revenda em função do preço de custo e a sazonalidade.
    - Entrega:
        - Dashboard interativo.
    - Local:
        - Plataforma cloud.  
    
- **Processo:**
    - Etapas:
        - Agrupar os dados por região.
        - Encontrar a mediana do preço de custo dos imóveis.
        - Identificar os imóveis que estão abaixo do preço mediano de cada região e que estejam em boas condições.
        - Coletar os dados de localização dos imóveis.
        - Agrupar os dados por estação do ano.
        - Definir o preço de revenda:
            - Se o preço de compra for maior que a mediana da região/sazonalidade, considerar uma margem de 10%.
            - Se o preço de compra for menor que a mediana da região/sazonalidade, considerar uma margem de 30%.
    - Formato da entrega:
        - Gráfico de dispersão indicando quais imóveis estão elegíveis para aquisição.
        - Tabela com os imóveis sugeridos para compra com os seguintes dados: código do imóvel, região, endereço, preço de custo, preço de venda sugerido, lucro previsto.
        - Mapa com a localização dos imóveis da tabela.
        - Filtro para selecionar uma região ou todas.
    - Local da entrega:
        - URL

- **Entrada:**
    - Fonte de dados:
        - Site Kaggle
    - Ferramentas:
        - Python
        - Streamlit framework
        - Heroku cloud
        - VSCode
        - Jupyter Notebook

## 3. Execução da solução

**Bibliotecas e parâmetros**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopy.geocoders
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import fixed, interact, interact_manual

from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim

# definindo formato de exibição dos números decimais
pd.set_option('display.float_format', lambda x: '%.2f' % x)

**Coleta dos dados**

In [ ]:
# dados dos imóveis em King County
raw_data = pd.read_csv('kc_house_data.csv')
raw_data.head()

In [ ]:
raw_data.dtypes

**Transformação dos dados**

In [ ]:
# corrigindo tipagem dos dados
raw_data['date'] = pd.to_datetime(raw_data['date'],format='%Y-%m-%d')
raw_data['bathrooms'] = raw_data['bathrooms'].astype('int64')
raw_data['floors'] = raw_data['floors'].astype('int64')

# removendo atributos que não agregam à solução do problema
raw_data = raw_data.drop(columns=['sqft_living15','sqft_lot15'])

# removendo registros duplicados
raw_data = raw_data.drop_duplicates(subset='id')

**Análise Descritiva**

In [ ]:
# separando dados de tipos numéricos
num_attributes = raw_data.select_dtypes(include=['int64','float64'])

# definindo medidas de dispersão e tendencia central
max_ = pd.DataFrame(num_attributes.apply(np.max))
min_ = pd.DataFrame(num_attributes.apply(np.min))
avg = pd.DataFrame(num_attributes.apply(np.mean))
median = pd.DataFrame(num_attributes.apply(np.median))
std = pd.DataFrame(num_attributes.apply(np.std))

# unindo resultados
descriptive_analysis = pd.concat([min_,max_,avg,median,std], axis=1).reset_index()
descriptive_analysis.columns = ['attributes','min','max','average','median','std']

In [ ]:
descriptive_analysis

**Limpeza dos dados**

In [ ]:
# correção de outlier
'''
assumo que o registro com 33 quartos é um erro de digitação,
considerando que os valores de preço, sala de estar, terreno
e numero de banheiros estão na média dos imóveis de 3 quartos
'''
raw_data[raw_data['bedrooms'].isin([3,33])]

price = raw_data[raw_data['bedrooms']==3]['price'].mean()
sqft_lot = raw_data[raw_data['bedrooms']==3]['sqft_lot'].mean()
sqft_living = raw_data[raw_data['bedrooms']==3]['sqft_living'].mean()
bath = raw_data[raw_data['bedrooms']==3]['bathrooms'].mean()

raw_data.loc[raw_data['bedrooms']==33,'bedrooms'] = 3

##### Levantamento de hipóteses

In [ ]:
# H1: Imóveis com vista para água são 20% mais caros que a média

_all = raw_data['price'].mean()
w_water = raw_data[raw_data['waterfront']==1]['price'].mean()
df_h1 = pd.DataFrame({'attribute': ['geral','waterfront'], 'valor_medio': [_all, w_water]})
df_h1['pct'] = df_h1['valor_medio'].pct_change()
result = ''
if df_h1.loc[1,'valor_medio'] > df_h1.loc[0,'valor_medio']:
    result = 'Verdadeiro'
else: 'Falso'

print('{:}. O valor dos imóveis com vista para a água apresentam variação de {:.2f}% em relação à média geral'.format(result, df_h1.loc[1,'pct']*100))
sns.barplot(data=df_h1, x='attribute', y='valor_medio');

In [ ]:
## H2: Imóveis com data de construção menor que 1955 são 50% mais baratos na média

_all = raw_data['price'].mean()
older_1955 = raw_data[raw_data['yr_built'] < 1955]['price'].mean()
df_h2 = pd.DataFrame({'attribute': ['media_geral','older_1995'], 'mean': [_all, older_1955]})
df_h2['pct'] = df_h2['mean'].pct_change()

print('Diferenca percentual: {:.2f}%'.format(df_h2.loc[1,'pct']*100))
sns.barplot(data=df_h2, x='attribute', y='mean');

In [ ]:
# H3: Imóveis sem porão são, em media, 40% maiores do que os imóveis com porão

sem_porao = raw_data[raw_data['sqft_basement']==0]['sqft_lot'].mean()
com_porao = raw_data[raw_data['sqft_basement']!=0]['sqft_lot'].mean()
df_h3 = pd.DataFrame({'attribute': ['com_porao','sem_porao'], 'mean_sqft_lot': [com_porao, sem_porao]})
df_h3['pct'] = df_h3['mean_sqft_lot'].pct_change()
result = ''
if df_h3.loc[0,'mean_sqft_lot'] < df_h3.loc[1,'mean_sqft_lot']:
    result = 'Verdadeiro'
else: result = 'Falso'

print('{:}. A diferença percentual da área total dos imóveis sem porão em relação aos imóveis com porão é de {:.2f}%'.format(result, df_h3.loc[1,'pct']*100))
sns.barplot(data=df_h3, x='attribute', y='mean_sqft_lot');

In [ ]:
# H4: Crescimento do preço dos imóveis Yoy é de 10%

raw_data['year'] = pd.to_datetime(raw_data['date']).dt.strftime('%Y')
df_h4 = raw_data[['year','price']].groupby('year').mean().reset_index()
df_h4 = df_h4.rename(columns={'price': 'mean'})
df_h4['pct'] = df_h4['mean'].pct_change()
result = ''
if df_h4.loc[1,'mean'] == 0.1:
    result = 'Verdadeiro'
else: result = 'Falso'
    
print('{:}. A variação percentual do valor médio dos imóveis ano a ano é de {:.2f}%'.format(result, df_h4.loc[1,'pct']*100))
sns.barplot(data=df_h4, x='year', y='mean');

In [ ]:
## H5: Imóveis com 3 banheiros tem um crescimento MoM de 15%

df_h5 = raw_data.loc[raw_data['bathrooms']==3,:].copy()
df_h5['yr_m'] = raw_data['date'].dt.strftime('%Y-%m')
df_h5 = df_h5[['yr_m','price']].groupby('yr_m').sum().reset_index()
df_h5 = df_h5.rename(columns={'price': 'mean'})
df_h5['pct'] = df_h5['mean'].pct_change()
df_h5['variacao'] = df_h5['pct'].apply(lambda x: 'positivo' if x>0 else 'negativo')

sns.set_style('darkgrid')
color = ['red','green']
p = sns.barplot(data=df_h5, x='yr_m', y='pct', hue='variacao', palette=color);
plt.xticks(rotation=45);

In [ ]:
## H6: Imóveis com sótão são 20% mais caros do que imóveis com porão

above = raw_data.loc[raw_data['sqft_above']>0,'price'].mean()
basement = raw_data.loc[raw_data['sqft_basement']>0,'price'].mean()
df_h6 = pd.DataFrame([['above',above],['basement',basement]],columns=['attribute','mean'])
df_h6['pct'] = df_h6['mean'].pct_change()

print('Diferenca percentual: {:.2f}%'.format(df_h6.loc[1,'pct']*100))
sns.barplot(data=df_h6, x='attribute', y='mean');

In [ ]:
# H7: Imóveis antigos reformados são 30% mais caros do que imóveis novos
    #imóvel antigo = ano de construção anterior a 2000

old = raw_data.loc[(raw_data['yr_renovated']>0) & (raw_data['yr_built'].between(1,2000)),'price'].mean()
new = raw_data.loc[(raw_data['yr_renovated']==0) & (raw_data['yr_built']>=2000),'price'].mean()
df_h7 = pd.DataFrame([['new',new],['old_renovated',old]],columns=['attribute','mean'])
df_h7['pct'] = df_h7['mean'].pct_change()
result = ''
if df_h7.loc[1,'mean'] > df_h7.loc[0,'mean']:
    result = 'Verdadeiro'
else: result = 'Falso'
    
print('{:}. A diferença da média do valor dos imóveis antigos reformados em relação aos imóveis novos é de {:.2f}%'.format(result, df_h7.loc[1,'pct']*100))
sns.barplot(data=df_h7, x='attribute', y='mean');

In [ ]:
# H8: Imóveis com vista pra água tem um crescimento YoY de 20%
df_h8 = raw_data[raw_data['waterfront']==1].copy()
df_h8 = df_h8[['year','price']].groupby('year').sum().reset_index()
df_h8 = df_h8.rename(columns={'price': 'mean'})
df_h8['pct'] = df_h8['mean'].pct_change()

print('Diferenca percentual: {:.2f}%'.format(df_h8.loc[1,'pct']*100))
sns.barplot(data=df_h8, x='year', y='mean');

In [ ]:
# H9: Imóveis com mais de 3 quartos são 30% mais caros que a média

_3more = raw_data.loc[raw_data['bedrooms']>3,'price'].mean()
_all = raw_data['price'].mean()
df_h9 = pd.DataFrame([['all',_all],['3more',_3more]],columns=['attribute','mean'])
df_h9['pct'] = df_h9['mean'].pct_change()

print('Diferenca percentual: {:.2f}%'.format(df_h9.loc[1,'pct']*100))
sns.barplot(data=df_h9, x='attribute', y='mean');

In [ ]:
# H10: Imóveis em boas condições com 1 andar são 20% mais caros que a média

raw_data['condition_type'] = raw_data['condition'].apply(lambda x: 'bad' if x <= 2 else 'regular' if x in([3,4]) else 'good' )
good1f = raw_data.loc[(raw_data['floors']==1)&(raw_data['condition_type']=='regular'),'price'].mean()
all = raw_data['price'].mean()
df_h0 = pd.DataFrame([['all',all],['good1f',good1f]], columns=['attribute','mean'])
df_h0['pct'] = df_h0['mean'].pct_change()

print('Diferenca percentual: {:.2f}%'.format(df_h0.loc[1,'pct']*100))
sns.barplot(data=df_h0, x='attribute', y='mean');

##### Construção do algoritmo da solução

**Engenharia de atributos**

In [ ]:
#new attributes
data = raw_data.copy()

data['sn_compra'] = 'NA'
data['condition_type'] = 'NA'

data['sell_price'] = 0.0
data['profit'] = 0.0
data['month'] = data['date'].dt.strftime('%m').astype('int64')
data['season'] = data['month'].apply(lambda x: 'winter' if x in(12,1,2) else
                                               'spring' if x in(3,4,5) else
                                               'summer' if x in(6,7,8) else 'fall' )
data['sn_compra'] = data['sn_compra'].astype('string')
data['condition_type'] = data['condition_type'].astype('string')
data['condition_type'] = data['condition'].apply(lambda x: 'good' if x==5 else 'regular' if x in(3,4) else 'bad')
data['season'] = data['season'].astype('string')

# definindo mediana do custo por região
regional_median = data[['zipcode','price']].groupby('zipcode').median().reset_index().copy()
regional_median.columns = ['zipcode','regional_median']

# definindo mediana do custo por região/estação
season_median = data[['price','zipcode','season']].groupby(['zipcode','season']).median('price')
season_median = season_median.rename(columns={'price': 'season_median'})

**Criando filtros**

In [ ]:
f_zipcode = widgets.Dropdown(
    options = data['zipcode'].sort_values().unique().tolist(),
    description = 'Zipcode',
    disable = False
)

f_filters = widgets.Checkbox(
    value=False,
    description='Disable filter',
    disabled=False,
    indent=False
)

**Definição dos imóveis para compra**

In [ ]:
# adicionando a mediana regional para cada imóvel
purchase = data.merge(regional_median,how='inner',on='zipcode').copy()

purchase['condition_type'] = purchase['condition_type'].astype('string')
purchase['sn_compra'] = purchase['sn_compra'].astype('string')

purchase['sn_compra'] = purchase.apply(
    lambda x: 'y' if (x['price'] < x['regional_median']) & (x['condition_type']=='good') else 'n',
    axis=1 )

purchase_list = purchase.drop(columns=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront',
                                  'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
                                  'yr_renovated', 'month'])

**Coleta dos dados de geolocalização dos imóveis elegíveis**

In [ ]:
# dados de geolocalização

geolocation = purchase_list.loc[purchase_list['sn_compra'] == 'y'][['id', 'lat', 'long']].copy().reset_index( drop=True )
geolocator = Nominatim( user_agent='geoapiExercises' )
geolocation['street'] = 'NA'
geolocation['house_num'] = 'NA'
geolocation['city'] = 'NA'
geolocation['neighbourhood'] = 'NA'
geolocation['state'] = 'NA'

for i in range( len( geolocation ) ):
    query = str( geolocation.loc[i, 'lat'] ) + ',' + str( geolocation.loc[i, 'long'] )
    response = geolocator.reverse( query, timeout=10000 )
    response = pd.json_normalize( response.raw['address'] )
    geolocation.iloc[i, 3] = response.apply( lambda x: x['road'] if 'road' in response.columns else 'NA', axis = 1 )    
    geolocation.iloc[i, 4] = response.apply( lambda x: x['house_number'] if 'house_number' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 5] = response.apply( lambda x: x['city'] if 'city' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 6] = response.apply( lambda x: x['neighbourhood'] if 'neighbourhood' in response.columns else
                                                       x['county'] if 'county' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 7] = response.apply( lambda x: x['state'] if 'state' in response.columns else 'NA', axis = 1 )
geolocation['address'] = geolocation['street'] + ', ' + geolocation['house_num']

**Definição do preço de revenda**

In [ ]:
sell = purchase_list.copy()

sell = sell.merge(season_median,how='inner',on=['zipcode','season'])
sell['sell_price'] = sell.apply(lambda x: x['price']*1.3 if x['price']<x['season_median'] else x['price']*1.1, axis=1)
sell['profit'] = sell['sell_price'] - sell['price']
sell = sell.loc[sell['sn_compra']=='y']
sell = sell.drop(columns=['date','sn_compra','condition_type','regional_median','lat','long'])

sell_list = sell.merge(geolocation[['id','address','neighbourhood','city']],how='inner',on='id')

In [ ]:
sell_list

**Visualização dos imóveis para compra**

In [ ]:
def data_viz( df, region ):
    plot = df[df['zipcode']==region][['id','price','zipcode','regional_median','condition_type','sn_compra']]
    plot['color'] = plot['sn_compra'].apply(lambda x: 'lightgray' if x == 'n' else 'blue')
     
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(y=plot['price'], 
                        marker_color=plot['color'],
                        mode='markers',
                        text='ID: '+plot['id'].astype('string')+' | Region: '+plot['zipcode'].astype('string')+' | Condition: ' + plot['condition_type'] + ' | Price: US$' + plot['price'].astype('string'),
                        name='Imovel'))

    fig.add_trace(go.Scatter(y=plot['regional_median'],
                        mode='lines',
                        text='Region: '+plot['zipcode'].astype('string')+' | Median: US$' + plot['regional_median'].astype('string'),
                        name='Mediana Regional'))
    
    fig.update_layout(
        xaxis_title="Houses",
        yaxis_title="Aquisition cost",
        font=dict( color="#000000" )
    )

    fig.show()
    
    return None

In [ ]:
interact(data_viz, df=fixed(purchase_list), region = f_zipcode);

**Visualização dos imóveis para compra/revenda no mapa**

In [ ]:
def make_map( house_list, geodata, region, filters ): 
    map_data = house_list[house_list['sn_compra']=='y'].merge(geodata[['id','address','neighbourhood','city']],how='inner',on='id').reset_index(drop=True)
    map_data = map_data.drop(columns=['date','sn_compra','condition_type','sell_price','profit','regional_median'])
    
    if filters:
        map_data
    else:
        map_data = map_data.loc[map_data['zipcode']==region]
    
    _map = px.scatter_mapbox( map_data,
                              lat='lat',
                              lon='long',
                              hover_name='address',                              
                              color='season',
                              color_discrete_sequence=px.colors.qualitative.Dark2,
                              zoom=10,
                              text='ID: '+map_data['id'].astype('string'),
                              hover_data=dict( lat=False, long=False, season=False)
                            )

    _map.update_layout(mapbox_style='open-street-map')
    _map.update_layout(height=600, margin = {'r':0,'t':0,'l':0,'b':0})
    _map.show()
    
    return None

In [ ]:
interact( make_map, house_list=fixed( purchase_list ), geodata=fixed( geolocation ), region=f_zipcode, filters=f_filters );